In [1]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.python.org/simple


In [2]:
import mne
import warnings


from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.exceptions import ConvergenceWarning

from mne import Epochs, pick_types, events_from_annotations
from mne.channels import make_standard_montage
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP

from utils.Pipeline import FourierTransform, Float64CSP, EEGStandardScaler
from utils.Preprocessing import load_and_preprocess_data


In [3]:
mne.set_log_level("CRITICAL")

# Suppress warnings from LogisticRegression and CSP
warnings.filterwarnings("ignore", category=UserWarning, module='sklearn.linear_model._logistic')
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='mne')

In [31]:
# Define subjects and runs for training and testing
train_subjects = [57, 22, 86]
train_tasks = {
    'Task 1': [3, 7, 11],
    'Task 2': [4, 8, 12],
    'Task 3': [5, 9, 13],
    'Task 4': [6, 10, 14]
}

test_subjects = [34]
test_tasks = {
    'Task 1': [3],
    'Task 2': [4],
    'Task 3': [5],
    'Task 4': [6]
}

In [32]:
logistic_regression = Pipeline([
    ('standarize', StandardScaler()),
    ('pca', PCA(n_components=10)),
    ('dslr', LogisticRegression(max_iter=10000))
])

display(logistic_regression)

Pipeline(steps=[('standarize', StandardScaler()), ('pca', PCA(n_components=10)),
                ('dslr', LogisticRegression(max_iter=10000))])

In [33]:
#task_1 (open and close left or right fist)
train_runs = train_tasks['Task 1']
train_epochs = load_and_preprocess_data(train_subjects, train_runs)

X_train = train_epochs.get_data().astype('float64')
y_train = train_epochs.events[:, -1]

display(X_train.shape)
display(y_train.shape)


test_runs = test_tasks['Task 1']
test_epochs = load_and_preprocess_data(test_subjects, test_runs)

X_test = test_epochs.get_data().astype('float64')
y_test = test_epochs.events[:, -1]

display(X_test.shape)
display(y_test.shape)

(195, 64, 481)

(195,)

(22, 64, 481)

(22,)